In [ ]:
# Distlearn Jupyter example

In [1]:
import client

In [2]:
c = client.Client("mike")

[2016-05-10 01:51:55,806] [INFO] This is client mike


INFO:client:This is client mike


[2016-05-10 01:51:55,812] [INFO] Connected to master


INFO:client:Connected to master


In [3]:
from sklearn.datasets import load_iris
from sklearn.cross_validation import train_test_split
iris = load_iris()
X = iris.data
Y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [4]:
c.send_data('iris_train', {'data': X_train, 'target': y_train})
c.send_data('iris_test', {'data': X_test, 'target': y_test})

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from dl_utils import Task
lr = LogisticRegression(C=1., penalty='l1')
task = Task(data='iris_train', test_data='iris_test', estimator=lr, 
            type='fit_predict', result='score', scoring=mean_squared_error)
task_id = c.send_task(task)
print(task_id, task.id)

(0, 0)


In [6]:
result = c.collect_task(task.id)
result

0.02

In [7]:
result = c.send_task(task, async=False)
result

0.02

In [8]:
from sklearn.cross_validation import StratifiedKFold
cv = StratifiedKFold(y_train, 5)
result = c.cross_validate(data='iris_train', estimator=lr, scoring=mean_squared_error, cv=cv, async=False)
result

[0.047619047619047616,
 0.10000000000000001,
 0.10000000000000001,
 0.050000000000000003,
 0.0]

In [10]:
from sklearn.tree import DecisionTreeClassifier
est1 = DecisionTreeClassifier(max_depth=10)
est2 = DecisionTreeClassifier(criterion='entropy', max_depth=2)
result = c.stacking(data='iris_train', estimators=[est1, est2], test_data='iris_test', 
                    estimator=lr, cv=cv, async=False)

In [11]:
mean_squared_error(y_test, result)

0.02

In [12]:
result = c.stacking(data='iris_train', estimators=[est1, est2], test_data='iris_test2', 
                    estimator=lr, cv=cv)

In [19]:
c.master.client_get_errors(c.id)

[]

In [14]:
c.send_data('iris_test2', {'data':X_test})

In [17]:
result = c.collect_task(result)

TypeError: %d format: a number is required, not numpy.ndarray

In [16]:
mean_squared_error(y_test, result)

0.02